In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_dynamics.jl")
# include("./funlopt/funl_constraint.jl")
# include("./trajopt/scaling.jl")

discretize_foh (generic function with 2 methods)

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/xuQYZ_unicycle_0328" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"]
Qnom = my_dict["Q"]
Ynom = my_dict["Y"]
Znom = my_dict["Z"]
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
ix = 3
iu = 2
iϕ = 2
iv = 2
iψ = iϕ*iv
iμ = iψ
@assert size(xnom,2) - 1 == N

In [4]:
dynamics = Unicycle()
decay_rate = 0.9
DLMI = NonlinearDLMI(decay_rate,ix,iu,dynamics.Cv,dynamics.Dvu)

NonlinearDLMI(0.9, 3, 2, 9, 6, 1, 9, 16, [0.0 0.0 1.0; 0.0 0.0 0.0], [0.0 0.0; 1.0 0.0], [1 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 1], [1 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 1])

In [5]:
idx = 4
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])

([0.0 0.0 -0.8563473835516268; 0.0 0.0 1.2805836524424632; 0.0 0.0 0.0], [0.831263029672674 0.0; 0.5558792814086592 0.0; 0.0 1.0])

In [6]:
include("./funlopt/funl_dynamics.jl")

discretize_foh (generic function with 2 methods)

In [7]:
idx = 4
# Q = Qnom[:,:,idx]
# K = Ynom[:,:,idx] * inv(Qnom[:,:,idx])
# Z = Znom[:,:,idx]
lam = 1.5
Q = randn(3,3)
K = randn(2,3)
Z = randn(3,3)
# Q = Q' + Q
X = vec(Q)
U = vcat(vec(K),vec(Z),lam)
Fx_n,Fu_n = diff_numeric(DLMI,X,U,A,B)
Fx,Fu = diff(DLMI,X,U,A,B)

([-5.443958518535371 0.20917421589151491 … 0.0 0.0; 4.0720557962353405 0.3157353960062347 … 0.0 0.0; … ; 0.0 0.0 … 0.3157353960062347 2.7724049189726627; 0.0 0.0 … 0.737652916730961 -0.2302225672458902], sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 3  …  9, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16], [-0.3095815761141645, 1.0900968242270404, 1.8625012227580835, 1.0900968242270404, -2.288640781712772, -3.9063346957702767, 1.8625012227580835, -3.9063346957702767, -6.6674561174065365, -0.10749451322758594  …  1.0, 0.597469247776017, -1.0915148723925099, -0.3558875950748849, -1.0915148723925099, 3.01892654477545, 2.860694000006466, -0.3558875950748849, 2.860694000006466, 4.979961798126741], 9, 16))

In [8]:
println(sum(abs.(Fx_n-Fx)))

2.4162988832010512e-9


In [9]:
println(sum(abs.(Fu_n-Fu)))

2.3887437583525184e-9


In [10]:
using BenchmarkTools

In [11]:
@benchmark diff_numeric(DLMI,X,U,A,B)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   95.375 μs …  11.194 ms  ┊ GC (min … max): 0.00% … 98.55%
 Time  (median):      99.250 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   109.822 μs ± 206.751 μs  ┊ GC (mean ± σ):  8.05% ±  5.30%

   ▃█▁ ▂▁                                                        
  ▃███▇██▇▆▆▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂ ▃
  95.4 μs          Histogram: frequency by time          137 μs <

 Memory estimate: 144.61 KiB, allocs estimate: 1530.

In [12]:
@benchmark diff(DLMI,X,U,A,B)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  18.292 μs …  11.656 ms  ┊ GC (min … max):  0.00% … 98.68%
 Time  (median):     30.250 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   45.651 μs ± 260.964 μs  ┊ GC (mean ± σ):  33.51% ±  6.81%

               ▁▆██▃▆▅▃▃▂▂▂▁                                    
  ██▇▆▄▄▅▄▃▃▄▅▆██████████████▆▅▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂ ▄
  18.3 μs         Histogram: frequency by time           58 μs <

 Memory estimate: 219.22 KiB, allocs estimate: 241.

In [13]:
Fu_n

9×16 Matrix{Float64}:
 -0.309582   0.0       -0.159773  …  0.0  0.0  0.0  0.0  0.0   0.597469
  1.0901     0.0       -1.14791      0.0  0.0  0.0  0.0  0.0  -1.09151
  1.8625    -0.107495   0.818265     0.0  0.0  0.0  0.0  0.0  -0.355888
  1.0901     0.0       -1.14791      0.0  0.0  0.0  0.0  0.0  -1.09151
 -2.28864    0.0        3.2106       1.0  0.0  0.0  0.0  0.0   3.01893
 -3.90633    0.619021   1.90952   …  0.0  1.0  0.0  0.0  0.0   2.86069
  1.8625    -0.107495   0.818265     0.0  0.0  1.0  0.0  0.0  -0.355888
 -3.90633    0.619021   1.90952      0.0  0.0  0.0  1.0  0.0   2.86069
 -6.66746    2.1165    -2.81553      0.0  0.0  0.0  0.0  1.0   4.97996

In [14]:
Fu

9×16 SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⎡⡧⡧⡧⠑⢄⠀⠀⢸⎤
⎢⣷⣷⣷⠀⠀⠑⢄⢸⎥
⎣⠉⠉⠉⠀⠀⠀⠀⠉⎦

In [15]:
DLMI.D

2×2 Matrix{Float64}:
 0.0  0.0
 1.0  0.0

In [16]:
Fu_n

9×16 Matrix{Float64}:
 -0.309582   0.0       -0.159773  …  0.0  0.0  0.0  0.0  0.0   0.597469
  1.0901     0.0       -1.14791      0.0  0.0  0.0  0.0  0.0  -1.09151
  1.8625    -0.107495   0.818265     0.0  0.0  0.0  0.0  0.0  -0.355888
  1.0901     0.0       -1.14791      0.0  0.0  0.0  0.0  0.0  -1.09151
 -2.28864    0.0        3.2106       1.0  0.0  0.0  0.0  0.0   3.01893
 -3.90633    0.619021   1.90952   …  0.0  1.0  0.0  0.0  0.0   2.86069
  1.8625    -0.107495   0.818265     0.0  0.0  1.0  0.0  0.0  -0.355888
 -3.90633    0.619021   1.90952      0.0  0.0  0.0  1.0  0.0   2.86069
 -6.66746    2.1165    -2.81553      0.0  0.0  0.0  0.0  1.0   4.97996

In [17]:
Fu

9×16 SparseMatrixCSC{Float64, Int64} with 60 stored entries:
⎡⡧⡧⡧⠑⢄⠀⠀⢸⎤
⎢⣷⣷⣷⠀⠀⠑⢄⢸⎥
⎣⠉⠉⠉⠀⠀⠀⠀⠉⎦

In [18]:
print_jl(Fx)
print_jl(Fu)

Type is Matrix{Float64}
Shape is (9, 9)
Type is SparseMatrixCSC{Float64, Int64}
Shape is (9, 16)


# constraint

In [19]:
idx = 4
# Q = Qnom[:,:,idx]
# K = Ynom[:,:,idx] * inv(Qnom[:,:,idx])
# Z = Znom[:,:,idx]
# lam = 1.3
Q = randn(3,3)
K = randn(2,3)
Z = randn(3,3)
lam = 1.5
X = vec(Q)
U = vcat(vec(K),vec(Z),lam)

16-element Vector{Float64}:
 -1.3719649622765238
 -1.3488043458991537
 -0.7148816619126837
  0.2646658488027596
 -0.3464937015057555
  1.8304540695113545
  1.1190810076031594
  0.7137492291231209
  0.03280920634611706
  0.9845725640700212
 -0.6599030139609467
  1.6437325069924313
 -1.5263841055921863
 -0.4419518747343653
 -1.7279893024466328
  1.5

In [20]:
a = [1.5;2.0]
function evaluate(q,k)
    Q = reshape(q,(3,3))
    K = reshape(k,(2,3))
    return a'*K*Q*K'*a
end

evaluate (generic function with 1 method)

In [21]:
evaluate(vec(Q),vec(K))

24.580244418840252

In [22]:
function eval_diff_numeric(q,k)
    ix = length(q)
    iu = length(k)

    eps_x = Diagonal{Float64}(I, ix)
    eps_u = Diagonal{Float64}(I, iu)
    fx = zeros(1,ix)
    fu = zeros(1,iu)

    h = 2^(-18)
    for i in 1:ix
        fx[:,i] .= (evaluate(q+h*eps_x[:,i],k) - evaluate(q-h*eps_x[:,i],k)) / (2*h)
    end
    for i in 1:iu
        fu[:,i] .= (evaluate(q,k+h*eps_u[:,i]) - evaluate(q,k-h*eps_u[:,i])) / (2*h)
    end
    return fx,fu
end

eval_diff_numeric (generic function with 1 method)

In [29]:
Cm = DLMI.Cm

6×6 Matrix{Int64}:
 1  0  0  0  0  0
 0  0  1  0  0  0
 0  0  0  0  1  0
 0  1  0  0  0  0
 0  0  0  1  0  0
 0  0  0  0  0  1

In [42]:
fk = vec(K)' * kron(Q, a*a') + (kron(Q, a*a') * vec(K))'
# fk = 2 * vec(K)' * kron(Q, a*a')
fq = kron(a'*K,a'*K)

1×9 adjoint(::Vector{Float64}) with eltype Float64:
 22.6153  2.58222  -14.938  2.58222  0.294839  …  -14.938  -1.70563  9.86693

In [43]:
fq_,fk_ = eval_diff_numeric(vec(Q),vec(K))

([22.61531415535137 2.5822232076898217 … -1.7056250860914588 9.866933808196336], [-11.887006116565317 -15.849341488443315 … 4.537731937132776 6.050309249665588])

In [44]:
println(sum(abs.(fq-fq_)))
println(sum(abs.(fk-fk_)))

9.1891166897895e-10
1.1433192170784423e-9


In [26]:
print_jl(fq_)
print_jl(fk_)

Type is Matrix{Float64}
Shape is (1, 9)
Type is Matrix{Float64}
Shape is (1, 6)
